In [ ]:
# --- Section spécifique à Colab (commentée pour l'exécution locale) ---
# !apt-get update && apt-get install -y tree

# Entraînement du Modèle Morningstar sur Google Colab (Adapté pour Local)

Ce notebook guide à travers les étapes nécessaires pour entraîner le modèle hybride multi-tâches Morningstar.
Il a été adapté pour fonctionner dans l'environnement local tout en conservant les commandes spécifiques à Colab en commentaires.

**Étapes principales :**
1. Configuration de l'environnement (installation des dépendances, exécution du pipeline de données).
2. Chargement et préparation des données pour un actif spécifique.
3. Définition et compilation de l'architecture du modèle Morningstar.
4. Entraînement du modèle avec suivi des performances.
5. Évaluation du modèle entraîné.
6. Visualisation des courbes d'apprentissage.

## 1. Configuration de l'Environnement

Nous devons d'abord installer les dépendances et générer les datasets via le pipeline.
Les commandes spécifiques à Colab pour cloner le dépôt sont commentées ci-dessous.

### 1.1 Clonage du Dépôt (Spécifique Colab - Commenté)

In [ ]:
# --- Section spécifique à Colab (commentée pour l'exécution locale) ---
# # Définir le répertoire de base pour le clonage
# REPO_DIR = "/content/CryptoRobot"
#
# # Supprimer le répertoire s'il existe déjà pour un clonage propre
# !rm -rf {REPO_DIR}
#
# # Cloner la branche spécifique 'mise-a-jour' du dépôt
# # Assurez-vous que le dépôt est accessible (public ou via token/clé SSH si privé)
# !git clone -b mise-a-jour https://github.com/Cabrel10/eva001.git {REPO_DIR}
#
# # Vérifier que le clonage a réussi
# !ls -l /content
# --- Fin de la section spécifique à Colab ---

### 1.2 Installation des Dépendances et Exécution du Pipeline

Cette cellule configure l'environnement en utilisant des chemins absolus basés sur la racine du projet.

In [ ]:
# --- Section spécifique à Colab (commentée pour l'exécution locale) ---
# # Se déplacer dans le répertoire cloné
# %cd {REPO_DIR} 
# --- Fin de la section spécifique à Colab ---

import os
from pathlib import Path

# --- Configuration Locale --- 
# Définir le chemin racine du projet
PROJECT_ROOT = Path('/home/morningstar/Desktop/crypto_robot/Morningstar') # Ajustez si nécessaire

# Vérifier le répertoire courant actuel (qui est 'notebooks/' au début de cette cellule)
!pwd

# Installer les dépendances depuis la racine du projet en utilisant le chemin absolu
print("\n--- Installation des dépendances ---")
requirements_path = PROJECT_ROOT / 'requirements.txt'
# Utiliser une variable pour le chemin dans la commande shell (avec guillemets pour la robustesse)
cmd_pip = f"pip install -r '{requirements_path}'"
print(f"Exécution: {cmd_pip}")
!{cmd_pip}

# Exécuter le pipeline de données complet depuis la racine du projet en utilisant le chemin absolu
print("\n--- Lancement du pipeline complet de données ---")
pipeline_script_path = PROJECT_ROOT / 'tests' / 'manual_tests' / 'test_full_pipeline_all_assets.py'
# Utiliser une variable pour le chemin dans la commande shell (avec guillemets)
# Note: L'exécution de scripts Python avec '!' utilise le PYTHONPATH du kernel, 
# donc l'ajout de PROJECT_ROOT au sys.path dans la cellule suivante est toujours nécessaire pour les imports DANS le script.
cmd_python = f"python '{pipeline_script_path}'"
print(f"Exécution: {cmd_python}")
!{cmd_python}

print("\n--- Pipeline de données terminé. Vérification des fichiers générés: ---")
# Vérifier le contenu du répertoire des données traitées en utilisant le chemin absolu
processed_data_path = PROJECT_ROOT / 'data' / 'processed'
# Utiliser une variable pour le chemin dans la commande shell (avec guillemets)
cmd_ls = f"ls -l '{processed_data_path}'"
print(f"Exécution: {cmd_ls}")
!{cmd_ls}

## 2. Chargement et Préparation des Données

Maintenant que les données sont générées dans `data/processed/` (relativement à la racine du projet), nous pouvons les charger.
Nous devons ajouter le répertoire racine du projet au `sys.path` pour que Python trouve les modules locaux (ex: `model.training.data_loader`).

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import sys
import os

# --- Configuration Locale --- 
# Définir le chemin racine du projet
PROJECT_ROOT = Path('/home/morningstar/Desktop/crypto_robot/Morningstar') # Ajustez si nécessaire

# Vérifier si le répertoire courant est la racine du projet
# Note: Le CWD peut avoir été modifié par %cd dans une cellule précédente si exécuté interactivement.
current_cwd = Path.cwd()
print(f"Répertoire courant actuel : {current_cwd}")
if current_cwd != PROJECT_ROOT:
    print(f"Attention: Le répertoire courant n'est pas la racine du projet ({PROJECT_ROOT}).")
    # Tentative de changement vers la racine du projet si on est dans 'notebooks'
    if current_cwd == PROJECT_ROOT / 'notebooks':
        try:
            print(f"Tentative de changement de répertoire vers {PROJECT_ROOT}...")
            os.chdir(PROJECT_ROOT)
            print(f"Répertoire courant changé pour : {Path.cwd()}")
        except FileNotFoundError:
            print(f"ERREUR: Impossible de trouver le répertoire projet {PROJECT_ROOT}")
    else:
        print("Les chemins relatifs pourraient ne pas fonctionner comme prévu.")

# Ajouter le répertoire racine du projet au PYTHONPATH pour les imports locaux
project_root_str = str(PROJECT_ROOT)
if project_root_str not in sys.path:
    print(f"Ajout de {project_root_str} au sys.path")
    sys.path.append(project_root_str)

# --- Section spécifique à Colab (commentée) ---
# # S'assurer que le répertoire courant est bien celui du projet
# PROJECT_ROOT_COLAB = "/content/CryptoRobot" # Doit correspondre au REPO_DIR ci-dessus
# if Path.cwd() != Path(PROJECT_ROOT_COLAB):
#     print(f"Changement du répertoire courant vers {PROJECT_ROOT_COLAB}")
#     os.chdir(PROJECT_ROOT_COLAB)
# !pwd # Confirmer
#
# # Ajouter le répertoire racine du projet au PYTHONPATH
# if PROJECT_ROOT_COLAB not in sys.path:
#     print(f"Ajout de {PROJECT_ROOT_COLAB} au sys.path")
#     sys.path.append(PROJECT_ROOT_COLAB)
# --- Fin Section Colab ---

# Importer le data_loader maintenant que le path est correct
try:
    from model.training.data_loader import load_and_split_data
except ModuleNotFoundError as e:
    print(f"ERREUR: Impossible d'importer 'model.training.data_loader'. Vérifiez:")
    print(f"  - La structure du projet dans {PROJECT_ROOT}")
    print(f"  - Que {PROJECT_ROOT} est bien dans sys.path: {sys.path}")
    print(f"  - L'erreur originale: {e}")
    raise # Arrêter l'exécution

# --- Configuration des Données --- 
ASSET_NAME = 'sol' # Choisir l'actif (btc, eth, sol, etc.)
# Chemin relatif à la racine du projet (maintenant que CWD et sys.path sont corrects)
DATA_DIR = PROJECT_ROOT / 'data' / 'processed' 
FILE_PATH = DATA_DIR / f"{ASSET_NAME}_final.parquet"

# --- IMPORTANT --- 
# Vérifiez que les colonnes générées par votre pipeline correspondent à celles-ci.
# Si l'erreur persiste, ajustez cette liste ou corrigez le pipeline.
LABEL_COLUMNS = ['trading_signal', 'volatility', 'market_regime']
VALIDATION_SPLIT = 0.2 # % des données pour la validation (fin de série)

print(f"Vérification de l'existence du fichier : {FILE_PATH}")
print(f"Existe: {FILE_PATH.exists()}")

print(f"Chargement des données pour : {ASSET_NAME} depuis {FILE_PATH}")

# Charger les données en tant que Tensors
X = None
y_dict = None
if FILE_PATH.exists():
    try:
        X, y_dict = load_and_split_data(FILE_PATH, label_columns=LABEL_COLUMNS, as_tensor=True)
        print(f"Données chargées : X shape={X.shape}, Labels={list(y_dict.keys())}")
    except ValueError as e:
        print(f"ERREUR lors du chargement/split : {e}")
        print("Vérifiez que les LABEL_COLUMNS ci-dessus correspondent aux colonnes dans le fichier Parquet.")
        # Pour inspecter les colonnes : df = pd.read_parquet(FILE_PATH); print(df.columns)
    except Exception as e:
        print(f"Une erreur inattendue est survenue lors du chargement : {e}")
else:
     print(f"ERREUR : Le fichier {FILE_PATH} n'a pas été trouvé. Vérifiez que le pipeline l'a bien généré pour cet actif et que le chemin est correct.")

# Continuer seulement si les données ont été chargées correctement
if X is not None and y_dict is not None:
    # Séparation Train/Validation (temporelle)
    num_samples = X.shape[0]
    num_val_samples = int(num_samples * VALIDATION_SPLIT)
    num_train_samples = num_samples - num_val_samples

    X_train, X_val = X[:num_train_samples], X[num_train_samples:]
    y_train_dict = {name: tensor[:num_train_samples] for name, tensor in y_dict.items()}
    y_val_dict = {name: tensor[num_train_samples:] for name, tensor in y_dict.items()}

    print(f"Séparation Train/Validation : Train={num_train_samples}, Val={num_val_samples}")
    print(f"Shapes : X_train={X_train.shape}, X_val={X_val.shape}")
    print(f"Labels Train : {[f'{k}:{v.shape}' for k, v in y_train_dict.items()]}")
    print(f"Labels Val : {[f'{k}:{v.shape}' for k, v in y_val_dict.items()]}")
else:
    print("\nArrêt prématuré car les données n'ont pas pu être chargées correctement.")
    # Assigner des valeurs vides pour éviter les erreurs dans les cellules suivantes si on les exécute quand même
    X_train, X_val, y_train_dict, y_val_dict = None, None, {}, {}

## 3. Définition et Compilation du Modèle Morningstar

In [ ]:
# Continuer seulement si les données d'entraînement existent
if X_train is not None:
    # Importer ici pour s'assurer que sys.path est correct
    try:
        from model.architecture.enhanced_hybrid_model import build_enhanced_hybrid_model
    except ModuleNotFoundError:
        print("ERREUR: Impossible d'importer 'model.architecture.enhanced_hybrid_model'. Vérifiez la structure.")
        raise

    # --- Configuration du Modèle --- 
    INPUT_SHAPE = (X_train.shape[1],) 
    # Doit correspondre à la cardinalité des labels de classification
    NUM_TRADING_CLASSES = 5 # Ex: Strong Sell -> Strong Buy
    NUM_REGIME_CLASSES = 3  # Ex: Bull, Bear, Sideways
    LEARNING_RATE = 0.001

    print("Construction du modèle Morningstar...")
    model = build_enhanced_hybrid_model(input_shape=INPUT_SHAPE, 
                                        num_trading_classes=NUM_TRADING_CLASSES, 
                                        num_regime_classes=NUM_REGIME_CLASSES)

    model.summary() # Afficher l'architecture

    # Définir les pertes et métriques (doivent correspondre aux noms des couches de sortie)
    # Assurez-vous que ces noms correspondent à ceux définis dans build_enhanced_hybrid_model
    losses = {
        'trading_signal_output': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        'volatility_output': tf.keras.losses.MeanSquaredError(),
        'market_regime_output': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    }
    metrics = {
        'trading_signal_output': ['accuracy'],
        'volatility_output': [tf.keras.metrics.RootMeanSquaredError(name='rmse'), 'mae'],
        'market_regime_output': ['accuracy']
    }
    # loss_weights = {'trading_signal_output': 1.0, 'volatility_output': 0.5, 'market_regime_output': 0.8} # Optionnel

    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

    print("Compilation du modèle...")
    model.compile(optimizer=optimizer, 
                  loss=losses, 
                  metrics=metrics)
                  # loss_weights=loss_weights)

    print("Modèle compilé.")
else:
    print("Définition/Compilation du modèle sautée car les données n'ont pas été chargées.")
    model = None # Pour éviter les erreurs suivantes

## 4. Entraînement du Modèle

In [ ]:
# Continuer seulement si le modèle est défini et les données existent
if model is not None and X_train is not None:
    # --- Configuration de l'Entraînement ---
    EPOCHS = 50
    BATCH_SIZE = 32
    # Utiliser le même ASSET_NAME que pour le chargement
    # Chemin relatif à la racine du projet
    MODEL_SAVE_DIR = PROJECT_ROOT / 'model' / 'training' 
    # Nom de fichier différent pour l'entraînement local
    MODEL_SAVE_PATH_LOCAL = MODEL_SAVE_DIR / f'{ASSET_NAME}_morningstar_local.h5'
    # Chemin Colab original (commenté)
    # MODEL_SAVE_PATH_COLAB = MODEL_SAVE_DIR / f'{ASSET_NAME}_morningstar_colab.h5'

    # Créer le répertoire de sauvegarde si nécessaire
    MODEL_SAVE_DIR.mkdir(parents=True, exist_ok=True)

    # Callbacks
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=MODEL_SAVE_PATH_LOCAL, # Utiliser le chemin local
        save_weights_only=False,
        monitor='val_loss', # Surveiller la perte totale de validation
        mode='min',
        save_best_only=True,
        verbose=1
    )
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=10, # Nb epochs sans amélioration avant arrêt
        verbose=1,
        restore_best_weights=True
    )

    print(f"Début de l'entraînement pour {EPOCHS} epochs...")
    print(f"Le meilleur modèle sera sauvegardé dans : {MODEL_SAVE_PATH_LOCAL}")
    # print(f"(Chemin Colab original commenté : {MODEL_SAVE_PATH_COLAB})")

    history = model.fit(
        X_train,
        y_train_dict,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val_dict),
        callbacks=[checkpoint_callback, early_stopping_callback],
        verbose=1
    )

    print("Entraînement terminé.")
else:
    print("Entraînement sauté car les données ou le modèle ne sont pas prêts.")
    history = None # Pour éviter les erreurs suivantes

## 5. Évaluation du Modèle

Évaluons les performances du modèle (avec les poids restaurés du meilleur epoch grâce à `restore_best_weights=True` dans EarlyStopping) sur l'ensemble de validation.

In [ ]:
# Continuer seulement si le modèle a été entraîné et les données de validation existent
if model is not None and X_val is not None:
    print("Évaluation du meilleur modèle sur l'ensemble de validation...")
    results = model.evaluate(X_val, y_val_dict, batch_size=BATCH_SIZE, verbose=0)

    print("Résultats de l'évaluation:")
    results_dict = {}
    try:
        for name, value in zip(model.metrics_names, results):
            results_dict[name] = value
            print(f"  - {name}: {value:.4f}")
    except AttributeError:
        print("Impossible de récupérer model.metrics_names, affichage brut:", results)

    # Optionnel : Charger explicitement le meilleur modèle sauvegardé si EarlyStopping n'a pas restauré les poids
    # print(f"\nChargement du meilleur modèle depuis {MODEL_SAVE_PATH_LOCAL} pour vérification...")
    # try:
    #    best_model = tf.keras.models.load_model(MODEL_SAVE_PATH_LOCAL)
    #    results_best = best_model.evaluate(X_val, y_val_dict, batch_size=BATCH_SIZE, verbose=0)
    #    print("Résultats du modèle chargé:")
    #    for name, value in zip(best_model.metrics_names, results_best):
    #        print(f"  - {name}: {value:.4f}")
    # except Exception as e:
    #    print(f"Erreur lors du chargement/évaluation du modèle sauvegardé: {e}")
    # except AttributeError:
    #    print("Impossible de récupérer model.metrics_names, affichage brut:", results_best)
else:
    print("Évaluation sautée.")

## 6. Visualisation des Courbes d'Apprentissage

Visualisons l'évolution des pertes et des métriques clés pendant l'entraînement.

In [ ]:
# Continuer seulement si l'entraînement a eu lieu
if history is not None:
    history_dict = history.history

    # Clés disponibles dans l'historique
    print("\nClés disponibles dans l'historique:", list(history_dict.keys()))

    # Créer les graphiques
    # Vérifier si les clés existent avant de plotter pour éviter les erreurs
    if 'loss' in history_dict and 'val_loss' in history_dict:
        epochs_range = range(1, len(history_dict['loss']) + 1)
        plt.figure(figsize=(15, 10))

        # 1. Perte Totale
        plt.subplot(2, 3, 1)
        plt.plot(epochs_range, history_dict['loss'], label='Train Loss')
        plt.plot(epochs_range, history_dict['val_loss'], label='Validation Loss')
        plt.title('Total Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # 2. Trading Signal Accuracy (si disponible)
        # Les noms peuvent varier légèrement selon la version de TF/Keras
        ts_acc_key = next((k for k in history_dict if k.startswith('trading_signal_output') and k.endswith('accuracy')), None)
        val_ts_acc_key = next((k for k in history_dict if k.startswith('val_trading_signal_output') and k.endswith('accuracy')), None)
        if ts_acc_key and val_ts_acc_key:
            plt.subplot(2, 3, 2)
            plt.plot(epochs_range, history_dict[ts_acc_key], label='Train Accuracy')
            plt.plot(epochs_range, history_dict[val_ts_acc_key], label='Validation Accuracy')
            plt.title('Trading Signal Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()
        else:
             print(f"Métriques 'trading_signal_output_accuracy' non trouvées (cherché: {ts_acc_key}, {val_ts_acc_key}).")

        # 3. Volatility RMSE (si disponible)
        vol_rmse_key = next((k for k in history_dict if k.startswith('volatility_output') and k.endswith('rmse')), None)
        val_vol_rmse_key = next((k for k in history_dict if k.startswith('val_volatility_output') and k.endswith('rmse')), None)
        if vol_rmse_key and val_vol_rmse_key:
            plt.subplot(2, 3, 3)
            plt.plot(epochs_range, history_dict[vol_rmse_key], label='Train RMSE')
            plt.plot(epochs_range, history_dict[val_vol_rmse_key], label='Validation RMSE')
            plt.title('Volatility RMSE')
            plt.xlabel('Epoch')
            plt.ylabel('RMSE')
            plt.legend()
        else:
             print(f"Métriques 'volatility_output_rmse' non trouvées (cherché: {vol_rmse_key}, {val_vol_rmse_key}).")

        # 4. Market Regime Accuracy (si disponible)
        mr_acc_key = next((k for k in history_dict if k.startswith('market_regime_output') and k.endswith('accuracy')), None)
        val_mr_acc_key = next((k for k in history_dict if k.startswith('val_market_regime_output') and k.endswith('accuracy')), None)
        if mr_acc_key and val_mr_acc_key:
            plt.subplot(2, 3, 4)
            plt.plot(epochs_range, history_dict[mr_acc_key], label='Train Accuracy')
            plt.plot(epochs_range, history_dict[val_mr_acc_key], label='Validation Accuracy')
            plt.title('Market Regime Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()
        else:
             print(f"Métriques 'market_regime_output_accuracy' non trouvées (cherché: {mr_acc_key}, {val_mr_acc_key}).")

        # 5. Volatility MAE (si disponible et intéressante)
        vol_mae_key = next((k for k in history_dict if k.startswith('volatility_output') and k.endswith('mae')), None)
        val_vol_mae_key = next((k for k in history_dict if k.startswith('val_volatility_output') and k.endswith('mae')), None)
        if vol_mae_key and val_vol_mae_key:
            plt.subplot(2, 3, 5)
            plt.plot(epochs_range, history_dict[vol_mae_key], label='Train MAE')
            plt.plot(epochs_range, history_dict[val_vol_mae_key], label='Validation MAE')
            plt.title('Volatility MAE')
            plt.xlabel('Epoch')
            plt.ylabel('MAE')
            plt.legend()
        else:
             print(f"Métriques 'volatility_output_mae' non trouvées (cherché: {vol_mae_key}, {val_vol_mae_key}).")

        plt.tight_layout()
        plt.show()
    else:
        print("Données de perte ('loss', 'val_loss') non trouvées dans l'historique, impossible de générer les graphiques.")
else:
    print("Visualisation sautée car l'entraînement n'a pas eu lieu.")

## Fin du Notebook

Le modèle a été entraîné (si les données étaient correctes), évalué, et le meilleur modèle a été sauvegardé localement. Les courbes d'apprentissage donnent un aperçu de la convergence et des éventuels problèmes (sur-apprentissage, etc.).

Pour utiliser sur Colab, décommentez les sections spécifiques à Colab et ajustez les chemins si nécessaire.